In [2]:
import numpy as np
from dppy.finite_dpps import FiniteDPP

r, N = 4, 10 #r number of features and N number of videos

#L-Ensemble of rank r that is not a projection DPP
Phi = np.random.randn(r, N)
DPP_L = FiniteDPP('likelihood', **{'L_gram_factor': Phi})

#Projection DPP with the corresponding previous eigenvectors
L = np.matmul(np.transpose(Phi), Phi)
e_vals, e_vecs = np.linalg.eigh(L)
proj_DPP = FiniteDPP('correlation', projection=True,
                    **{'K_eig_dec': (np.concatenate((np.zeros(N-r),np.ones(r))), e_vecs)})

L_dual = Phi Phi.T was computed: Phi (dxN) with d<N


In [3]:
for _ in range(10):
    print(DPP_L.sample_exact())

[9, 7]
[9, 7, 8]
[3, 2, 8]
[4, 5, 8, 2]
[0, 8, 7]
[8, 9, 6]
[0, 4, 5]
[8, 4, 0, 9]
[9, 4]
[9, 0]


In [4]:
for _ in range(10):
    print(proj_DPP.sample_exact())

[9, 7, 0, 1]
[0, 1, 9, 8]
[6, 0, 8, 9]
[8, 9, 0, 6]
[9, 3, 5, 8]
[9, 0, 1, 2]
[4, 2, 9, 0]
[8, 4, 5, 9]
[5, 4, 8, 2]
[8, 4, 9, 5]


In [6]:
#Compute singleton marginal of r-DPP using formula (205) of 'DPP for Machine Learning'
from dppy.exact_sampling import elementary_symmetric_polynomials
e_N_r = elementary_symmetric_polynomials(e_vals, r)[r, N]
e_minus_n_r_minus_one = np.array(np.zeros(N))

for n in range(N):
    e_minus_n_r_minus_one[n] = elementary_symmetric_polynomials(np.concatenate((e_vals[:n], e_vals[n+1:])), r-1)[r-1, N-1]

singleton_marginals = np.array(np.zeros(N))
for i in range(N):
    for n in range(N):
        singleton_marginals[i] += e_vecs[i,n]**2*e_vals[n]*e_minus_n_r_minus_one[n]/e_N_r

print("Singleton marginals of r-DPP")
print(singleton_marginals)
print('\n')

print("Singleton marginals of associated projection DPP")
proj_DPP.compute_K()
print([proj_DPP.K[i,i] for i in range(N)])
print('\n')

print('Singleton marginals of L-Ensemble')
DPP_L.compute_K()
print([DPP_L.K[i,i] for i in range(N)])
print('\n')

Singleton marginals of r-DPP
[0.46237768 0.19153545 0.14362018 0.35755405 0.3285624  0.57936299
 0.36354747 0.10670674 0.72441855 0.74231448]


Singleton marginals of associated projection DPP
K (correlation) kernel computed via:
- U diag(eig_K) U.T
[0.46237768432712295, 0.19153544729876737, 0.1436201799401688, 0.35755405033623044, 0.3285624024915734, 0.5793629943195351, 0.3635474693958252, 0.1067067424265473, 0.7244185464436639, 0.7423144830205672]


Singleton marginals of L-Ensemble
K (correlation) kernel computed via:
- U diag(eig_K) U.T
[0.39764169010097417, 0.14810759300331378, 0.09795840892058154, 0.2766521422307766, 0.27411568458328583, 0.4655643423355296, 0.22148254401284057, 0.09288332535467797, 0.5964582075625253, 0.5749238428848871]




In [7]:
proj_DPP_singleton = np.array([proj_DPP.K[i,i] for i in range(N)])
L_ens_singleton = np.array([DPP_L.K[i,i] for i in range(N)])

print(proj_DPP_singleton/L_ens_singleton)

[1.16279982 1.29321828 1.46613427 1.29243189 1.19862679 1.24443163
 1.64142719 1.1488256  1.21453362 1.29115272]
